<a href="https://colab.research.google.com/github/kyanite153/Prog2kakushin/blob/main/work1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#大富豪（トランプゲーム）

#ソースコード

In [1]:
import pandas as pd
import random
'''
（メモ）
d
h
c
s
joker
'''
class CareerPoker:
  def __init__(self, name='player1'):
    self.player_name = name
    self.games = 3
    self.number_of_players = 4
    self.istrump_8 = True
    self.istrump_s3 = True
    self.isreg_sign = True
    self.ismiyakooti = True
    self.data = pd.DataFrame([[name, 0, 0, 0.]], columns=['名前', '1位', '総試合数', '勝率'])
  def is_play_card(self, player, cards=None):
    """
    cardsはset型で受け取る．
    """
    if not cards in self.get_all_card():
      return False
    a = [self.get_card_strength(x) for x in list(cards)]
    b = [a[x+1]-a[x] for x in range(len(a)-1)]
    if set(b)=={1}:
      return '階段3'
  def sort_cards(self, cards):
    return sorted(cards)
  def set_game(self):
    while True:
      print('確認/変更したいものを選択．')
      print('1:ラウンド数')
      print('2:8切り')
      print('3:スぺ３返し')
      print('4:記号しばり')
      print('5:都落ち')
      print('6:設定を終了する')
      n = int(input('ここに数字を入力:'))
      if n == 1:
        print(f'現在の設定：{self.games}')
        n = input('ラウンド数：')
        if 1<=n:
          self.games = n
        else:
          print('エラー．ラウンド数は1以上．')
      elif n == 6:
        break
  def get_card_strength(self, card):
    l = ['3','4','5','6','7','8','9','10','J','Q','K','A','2','oker']
    return l.index(card[1:])
  def get_all_card(self):
    l = ['3','4','5','6','7','8','9','10','J','Q','K','A','2']
    return {f'h{x}' for x in l}|{f'd{x}' for x in l}|{f's{x}' for x in l}|{f'c{x}' for x in l}|{'joker'}
  def run_game(self):
    com = [CareerPokerCOM(self, f'com{x+1}') for x in range(self.number_of_players-1)]
    players = [self.player_name]+com
    score_data = pd.DataFrame([[players[j]]+[None for i in range(self.games)] for j in range(self.number_of_players)], columns=['name']+[f'{x}試合目' for x in range(1,self.games+1)])
    print(score_data)
    all_card = self.get_all_card()
    self.state = {players[i]:{} for i in range(self.number_of_players)}
    self.state['deck'] = all_card
    self.cards_select = None
    #始めの親を決める
    parent = random.choice(players)
    while True:
      #ゲーム開始
      print(f'{parent}の番です．')
      break

class CareerPokerCOM:
  def __init__(self, master, name='com'):
    self.strength = 1 #1~3
    self.master = master
    self.name = name
  def play_card(self):
    pass
  def __repr__(self):
    return self.name

def main():
  global master
  master=CareerPoker()
  master.run_game()

if __name__ == '__main__':
  main()


      name  1試合目  2試合目  3試合目
0  player1  None  None  None
1     com1  None  None  None
2     com2  None  None  None
3     com3  None  None  None
player1の番です．


In [2]:
master.is_play_card('deck',{'d2'})


False

In [ ]:
master.state

{'player1': {},
 com1: {},
 com2: {},
 com3: {},
 'deck': {'c10',
  'c2',
  'c3',
  'c4',
  'c5',
  'c6',
  'c7',
  'c8',
  'c9',
  'cA',
  'cJ',
  'cK',
  'cQ',
  'd10',
  'd2',
  'd3',
  'd4',
  'd5',
  'd6',
  'd7',
  'd8',
  'd9',
  'dA',
  'dJ',
  'dK',
  'dQ',
  'h10',
  'h2',
  'h3',
  'h4',
  'h5',
  'h6',
  'h7',
  'h8',
  'h9',
  'hA',
  'hJ',
  'hK',
  'hQ',
  'joker',
  's10',
  's2',
  's3',
  's4',
  's5',
  's6',
  's7',
  's8',
  's9',
  'sA',
  'sJ',
  'sK',
  'sQ'}}